# [二版] - 去 英數字 符號 逗號 , 且可選是否印標題
# 實作 - 寫入檔案
# 萃取字詞的四大方法
# 首先要先確定有 [字典] 和 [停用詞] 的txt檔

In [1]:
import os
import jieba
import jieba.analyse
import jieba.posseg as pseg  
jieba.set_dictionary('dict.big.tra.txt')  #預設字典
jieba.load_userdict('dict_twstd_tfidf.txt')  #中文分詞詞庫(TFIDF)
jieba.load_userdict('dict_ntusd_pos.txt') #NTUSD_負向
jieba.load_userdict('dict_ntusd_nag.txt') #NTUSD_正向
jieba.load_userdict('dict_twedu_dict.txt') #教育部《重編國語辭典 修訂本》


#jieba.load_userdict('dict_20160111_NotReallyUseful.txt')  
#中文分詞詞庫(TFIDF) + 百度分詞詞庫 + 69萬中文大辭典 + 中文热门词库(155073) 
#不是很好用，可備著使用


jieba.analyse.set_stop_words("stopwords_0104_test.txt") #停用詞

Building prefix dict from C:\Users\BigData\dict.big.tra.txt ...
DEBUG:jieba:Building prefix dict from C:\Users\BigData\dict.big.tra.txt ...
Loading model from cache c:\users\bigdata\appdata\local\temp\jieba.ud8ec30fabaf60e161f06b3552aab2f0e.cache
DEBUG:jieba:Loading model from cache c:\users\bigdata\appdata\local\temp\jieba.ud8ec30fabaf60e161f06b3552aab2f0e.cache
Loading model cost 0.266 seconds.
DEBUG:jieba:Loading model cost 0.266 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In [2]:
#法一
def jiebaNoEng(attraction,rankNum,filename,title):
    import re
    f = open(attraction,'r')  #讀取檔案
    f1 = f.read()  #接住
    f2 = ''.join(f1.split())  #把空白全去掉
    words=jieba.cut(f2)  #斷詞
    
    li = []
    for u in words:
        if not re.search('(\w)(\w)',u):
            li.append(u)
    
    n1 = (' '.join(li)).encode('utf-8')
    f.close()
    x = jieba.analyse.extract_tags(n1, topK=rankNum, withWeight=True)    
    fid = open(filename,'a')
    
    #*#*#*#*#*#*#*#*#*#
    #決定是否要印景點標題#

    #print attraction   
    #fid.write(attraction)
    #fid.write('\n')
    
    TitleDic=['True','T','t','Yes','Y','y']
    if title in TitleDic:
        fid.write(attraction)
        fid.write('\n')
    else:
        fid.write('')
        
    #*#*#*#*#*#*#*#*#*#
    
    for i in x:
        fid.write(i[0].encode('utf-8')+'\n')
        #print i[0].encode('utf-8'),i[1]
    fid.close()
def mutijiebaNoEng(date,attrlist,RankNum,title):
    filename = 'NE_JiebaTfidf-'+str(RankNum)+'-'+date+'.txt'    
    for i in range(0,len(attrlist)):
        jiebaNoEng(attrlist[i],RankNum,filename,title)


################################################

################################################        
 
    
#法二
def wordsToList(attfile):
    import re
    f = open(attfile,'r')
    f1 = f.read()
    f2 = ''.join(f1.split())
    words=jieba.cut(f2)
    
    li = []
    for u in words: #把"有出現兩個連續英數字"的詞語濾掉
        if not re.search('(\w)(\w)',u):
            li.append(u)
    n1 = (' '.join(li)).encode('utf-8')
    return n1
    f.close()

def stopWordsList(stopwordfile):
    li = []
    fid =  open(stopwordfile,'r')
    for line in fid:
        li.append(line.strip().decode('utf-8'))#要解開utf-8,才能成功給scikitlearn用
    fid.close()
    return li

def SciNoEng(date,listInput,stopWordsFile,rankNum,title):
    from sklearn import feature_extraction  
    from sklearn.feature_extraction.text import TfidfTransformer  
    from sklearn.feature_extraction.text import CountVectorizer
    filename = 'NE_SciTfidf-'+str(rankNum)+'-'+date+'.txt'
    
    #創造一個空矩陣
    corpus = []
    
    #把傳入的List依序讀出，使用方法把這些東西全都讀出來
    for each in range(0,len(listInput)):
        corpus.append(wordsToList(listInput[each]))

    vectorizer=CountVectorizer(analyzer='word',stop_words=stopWordsList(stopWordsFile))  #該類會將文本中的詞語轉換為詞頻矩陣，矩陣元素a[i][j] 表示j詞在i類文本下的詞頻  
    x = vectorizer.fit_transform(corpus)  #將文本轉為詞頻矩陣 
    transformer=TfidfTransformer()  #該類會統計每個詞語的tf-idf權值 
    tfidf=transformer.fit_transform(x)  #計算tf-idf
    word=vectorizer.get_feature_names()  #獲取詞袋模型中的所有詞語
    weight=tfidf.toarray()   #將tf-idf矩陣抽取出來，元素a[i][j]表示j詞在i類文本中的tf-idf權重  
    
    with open(filename,'a') as files:
        import operator
        for i in range(len(weight)):
            
            #*#*#*#*#*#*#*#*#*#
            #決定是否要印景點標題#
            
            #print listInput[i]
            #files.write(listInput[i])
            #files.write('\n')
            
            TitleDic=['True','T','t','Yes','Y','y']
            if title in TitleDic:
                files.write(listInput[i])
                files.write('\n')
            else:
                files.write('')
            
            #*#*#*#*#*#*#*#*#*#
            dic = {}
            for j in range(len(word)):  
                dic[word[j]] = weight[i][j]
            words_freq = sorted(dic.iteritems(),key=operator.itemgetter(1),reverse=True)
            num2 = 0
            for ele in words_freq[0:rankNum]:
                num2 += 1
                if not ele[1] == float(0.0):
                    #print num2,"\t",ele[0],"\t",ele[1]
                    files.write(ele[0].encode('utf-8'))
                    files.write("\n")

                    
################################################  

################################################


#法三
def jiebafreNoEng(attraction,rankNum,filename,title):
#轉換象山資料,並切詞,並使用JIEBA內建的tfidf統計排名    
    import re
    f = open(attraction,'r')  #讀取檔案
    f1 = f.read()  #接住
    f2 = ''.join(f1.split())  #把空白全去掉
    words=jieba.cut(f2)  #斷詞
    
    #清洗
    li = []
    dot = ['。','（','）','！','「','」','，','、','；','：','”','“','～'\
           'Ｐ','Ｓ','．','é','︶','』','『','﹗','ī','ō']
    for u in words:
        if not re.search('(\w)(\w)',u): #清除連續兩個英文字
            if not re.match('([#%@$.,/!()-;~&\'\"]|[\w]|[*^?><])',u): #清除標點符號和單一英文字符
                if u.encode('utf-8') not in dot:  #清洗全形符號
                    li.append(u)
    
    n1 = (' '.join(li)).encode('utf-8')
    f.close()
    x = jieba.analyse.extract_tags(n1, topK=rankNum, withWeight=True)    
    fid = open(filename,'a')
    
    #統計字頻
    dic={}
    for w in n1.split(' '):
        if w not in dic:
            dic[w] = 1
        else:
            dic[w] = dic[w] + 1
    import operator
    words_freq = sorted(dic.iteritems(),key=operator.itemgetter(1),reverse=True)
    #print attraction#,"\n總詞量：",len(words_freq)

    #*#*#*#*#*#*#*#*#*#
    #決定是否要印景點標題#
            
    #print attraction
    #fid.write(attraction)
    #fid.write('\n')
    
    TitleDic=['True','T','t','Yes','Y','y']
    if title in TitleDic:
        fid.write(attraction)
        fid.write('\n')
    else:
        fid.write('')
    
    #*#*#*#*#*#*#*#*#*#
    
    
    #列出原始字頻排名
    #print "==========字頻統計=========="
    #print "排名","\t","詞彙","\t","字頻"
    #print "- - - - - - - - - - - - - - "
    num2 = 0
    for ele in words_freq[0:rankNum]:
        num2 += 1
        #print num2,"\t",ele[0],"\t",ele[1]
        #print ele[0]
        #print type(ele[0])
        #x = ele[0].encode('utf-8')
        fid.write(ele[0])        
        fid.write('\n')
    fid.close()
#        if num2 % 10 == 0:
            #print "- - - - - - - - - - - - - - "
            
# 比對多個景點斷詞字頻的方法
def mutifreNoEng(date,attrlist,RankNum,title):
    filename = 'NE_WordsFre-'+str(RankNum)+'-'+date+'.txt'  
    for i in range(0,len(attrlist)):
        jiebafreNoEng(attrlist[i],RankNum,filename,title)

        
################################################  

################################################


#法四
def stopWordsSet(stopwordfile):
    li = []
    fid =  open(stopwordfile,'r')
    for line in fid:
        li.append(line.strip())
    fid.close()
    setli = set(li)
    return setli

def jiebafrestpNoEng(attraction,stopwordfile,rankNum,filename,title):
    import re
    f = open(attraction,'r')  #讀取檔案
    f1 = f.read()  #接住
    f2 = ''.join(f1.split())  #把空白全去掉
    words=jieba.cut(f2)  #斷詞
    
    li = []
    dot = ['。','（','）','！','「','」','，','、','；','：','”','“','～',\
           'Ｐ','Ｓ','．','é','︶','』','『','﹗','ī','ō','Ｉ','Ｍ']
    dot = set(dot)
    for u in words:
        if not re.search('(\w)(\w)',u):
            if not re.search('([#%@$.,/!()-;~&\'\"]|[\w]|[*^?><])',u):
                if u.encode('utf-8') not in dot:
                    li.append(u)
    
    n1 = (' '.join(li)).encode('utf-8')
    f.close()
    
    fid = open(filename,'a')
    
    #統計字頻
    dic={}
    comparelist = stopWordsSet(stopwordfile)
    for w in n1.split(' '):
        if w not in comparelist:
            if w not in dic:
                dic[w] = 1
            else:
                dic[w] = dic[w] + 1
    import operator
    words_freq = sorted(dic.iteritems(),key=operator.itemgetter(1),reverse=True)

    #*#*#*#*#*#*#*#*#*#
    #決定是否要印景點標題#
            
    #print attraction
    #fid.write(attraction)
    #fid.write('\n')
    
    TitleDic=['True','T','t','Yes','Y','y']
    if title in TitleDic:
        fid.write(attraction)
        fid.write('\n')
    else:
        fid.write('')
    
    #*#*#*#*#*#*#*#*#*#
           
    num2 = 0
    for ele in words_freq[0:rankNum]:
        num2 += 1
        fid.write(ele[0])
        fid.write('\n')
    fid.close()        
# 比對多個景點斷詞字頻(去停用詞後)的方法
def mutifrestpNoEng(date,attrlist,stopWordsFile,RankNum,title):
    filename = 'NE_WordsFreAddStop-'+str(RankNum)+'-'+date+'.txt' 
    for i in range(0,len(attrlist)):
        jiebafrestpNoEng(attrlist[i],stopWordsFile,RankNum,filename,title)




################################################  

################################################
#綜合比對


def FourWayPK(folder,date,rank,stpDic,title):
    #實作：
    import time
    import os
    lis = []
    for name in os.listdir(folder):
        path = folder+'\\'+name
        lis.append(path)

    x = time.time()

    mutijiebaNoEng(date,lis,rank,title)

    SciNoEng(date,lis,stpDic,rank,title)

    mutifreNoEng(date,lis,rank,title)

    mutifrestpNoEng(date,lis,stpDic,rank,title)

    y = time.time()
    print 'Finish!'
    print 'Cost ',y-x,' secs.'

# 綜合比對 方法
使用說明：

    FourWayPK(資料夾,日期,排名,停用詞字典,標題)
    
    資料夾：[字串]。資料夾名稱，內含所有以景點為單位的TXT檔(命名要用英文)
           EX:有一'AttrSample'的資料夾，內有30個檔，
              每個檔都是一個單獨景點的所有評論\遊記。
              
    日　期：[字串]。輸入今天日期，格式-->'YYYYMMDD'
           EX:'20151223'
           
    排　名：[整數]。想要顯示各方法的前幾個排名數
           EX:10
           
    停用詞：[字串]。TXT檔。內容格式需為一行一個字詞，字詞後不得有逗點
           EX:'stopwords_1222.txt'
           
    標　題：[字串]。選擇是否顯示標題(各景點名稱)
           傳入：'T'　顯示
           　　　'F'　不顯示，檔案裡只會有全部詞語

方法結果：

    會在所在的資料夾產生四個TXT檔，開頭分別為：
    1.NE_JiebaTfidf　　　　-->單一TFIDF
    2.NE_SciTfidf         -->多重TFIDF
    3.NE_WordsFre         -->普通字頻統計
    4.NE_WordsFreAddStop  -->去停用詞字頻統計
    
    分別是四個方法的結果，並且會依照景點排序。
    
    且會傳回所耗時數！
    
# 【注意】每次使用的＂日期＂都要設不一樣
# 　　　　否則會在同一個檔案一直append下去

In [3]:
FourWayPK('AttrSample100','20160106',30,'stopwords_0104_test.txt','T')

Finish!
Cost  30.896999836  secs.
